In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
## Loading trained model
model = load_model("Ann_model.h5")

## loading the encoder and scaler
with open("onehotencoder_geo.pkl", "rb") as file:
    onehotencoder_geo = pickle.load(file)

with open("gender_encoder.pkl", "rb") as file:
    gender_encoder = pickle.load(file)

with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

    





In [17]:
## Example input data
input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000

}

In [18]:
## One-hot encode "geography" column
geo_encoder = onehotencoder_geo.transform([[input_data["Geography"]]]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoder, columns = onehotencoder_geo.get_feature_names_out(["Geography"]))
geo_encoder_df


d:\Ann_Classification_project\venvANN\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [19]:
## Converting the input data (dictinary) into pandas dataframe
input_df= pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [20]:
## Converting categorical columns (gender) using saved gender_encoder
input_df["Gender"] = gender_encoder.transform(input_df["Gender"])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [21]:
## Converting categorical column (geography) using saved onehotencoder_geo
input_df = pd.concat([input_df.drop("Geography", axis = 1),geo_encoder_df], axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [22]:
## Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.51697034,  0.90339252,  0.10318316, -0.69357359, -0.26460877,
         0.80940927,  0.64959174,  0.97725852, -0.86250858,  1.        ,
        -0.57715782, -0.57754272]])

In [23]:
## Prediction of Churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 814ms/step


array([[0.01283627]], dtype=float32)

In [26]:
prediction_probability = prediction[0][0]


In [27]:
if prediction_probability > 0.5:
    print("The Customer is likely to Churn")
else:
    print("The Customer is not likely to Churn")

The Customer is not likely to Churn
